In [2]:
from transformers import T5Tokenizer, T5EncoderModel
from datasets import load_dataset
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
import torch
# 加载数据集
dataset = load_dataset("yizhongw/self_instruct", 'self_instruct', split='train[:5000]')  # 只取前100个样本进行测试

# 加载 T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained('google/umt5-small')
model = T5EncoderModel.from_pretrained('google/umt5-small')

# 对数据集文本进行编码
inputs = tokenizer(dataset['prompt'], return_tensors="pt", padding=True, truncation=True, max_length=512)
input_ids = inputs.input_ids

# 获取 T5 模型的嵌入
all_embeddings = []
with torch.no_grad():
    outputs = model(input_ids=input_ids)
    embeddings = outputs.last_hidden_state
    # embeddings = outputs.last_hidden_state.squeeze(0)
    # all_embeddings.append(embeddings)

# 使用嵌入的平均值作为特征
features = embeddings.mean(dim=1).numpy()

# all_embeddings_tensor = torch.cat(all_embeddings, dim=0)
# print(all_embeddings_tensor.shape)
# all_embeddings_flattened = all_embeddings_tensor.view(-1, all_embeddings_tensor.size(-1))
# print(all_embeddings_flattened.shape)


# 应用 t-SNE
tsne = TSNE(n_components=2, random_state=0)
X_tsne = tsne.fit_transform(features)
print(X_tsne.shape)

# from sklearn.cluster import KMeans

# # 假设 X_tsne 是 t-SNE 的结果
# num_clusters = 5  # 指定想要的聚类数量

# # 进行 K-Means 聚类
# kmeans = KMeans(n_clusters=num_clusters, random_state=0)
# clusters = kmeans.fit_predict(X_tsne)



from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

# 假设 X_tsne 是您的 t-SNE 降维后的数据
# 应用 DBSCAN 聚类
# eps 和 min_samples 参数可能需要根据您的数据进行调整
dbscan = DBSCAN(eps=0.5, min_samples=5)
clusters = dbscan.fit_predict(X_tsne)


# 可视化聚类结果
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=clusters, cmap='viridis')
plt.title("t-SNE with K-Means Clustering")
plt.xlabel("t-SNE Feature 1")
plt.ylabel("t-SNE Feature 2")
plt.colorbar()  # 显示颜色条
plt.show()

: 